In [ ]:
import os
import sys
from datetime import datetime
from time import sleep
from selenium import webdriver
import configparser
import database_functions as dbf
import locale
import re

In [ ]:
#IMPORT CONFIG FROM FILE .CONFIG
config_file_name = 'price_monitoring.config'
locale.setlocale(locale.LC_ALL, "ko_KR.UTF-8")
#cwd = os.path.dirname(sys.argv[0]) #working on linux server or windows python script
cwd = os.getcwd() #working on windows during python notebook execution
cfg = configparser.ConfigParser()
ini_config_path = os.path.join(cwd,config_file_name)
cfg.read(ini_config_path)
driver_path_chrome = cfg['general_config']['driver_path_chrome']
driver_path_edge = cfg['general_config']['driver_path_edge']
database = cfg['database_config']['database']

link_prefix = cfg['general_config']['link_prefix']
link_sufix = cfg['general_config']['link_sufix']
link_sufix_split = cfg['general_config']['link_sufix_split']



In [ ]:
#FUNCTION TO GET PRICE FROM WEBPAGE
def get_price(driver):
    xpath_price = '/html/body/div[2]/section/div[1]/div/div[3]/div[5]/div[1]/div/div[2]/span[1]/strong'
    price_content = driver.find_element_by_xpath(xpath_price).text
    try:
        price_content = price_content.replace('원', '')
    except:
        price_content = price_content
    price = locale.atof(price_content)
    return price

#FUNCTION TO GET PRICE FROM WEBPAGE
def get_image(driver):
    xpath_image = '/html/body/div[2]/section/div[1]/div/div[1]/div[1]/img'
    image_content = driver.find_element_by_xpath(xpath_image).get_attribute('src')
    return image_content


In [ ]:
#CONNECT TO DATABASE AND CREATE TABLE IF IT DOESN'T EXIST
conn, cur = dbf.connect_to_database(database)
columns = "price REAL, time_utc_now TEXT, image_link TEXT, product_id TEXT, product_link TEXT"
table_name = "prices"
dbf.create_table(conn, cur, table_name, columns)


In [ ]:
#URL TO SCRAPE
url_list = [
    r'https://www.coupang.com/vp/products/6366671555?isAddedCart=',
    r'https://www.coupang.com/vp/products/5056493492?isAddedCart=',
    r'https://www.coupang.com/vp/products/5835511110?isAddedCart=',
    r'https://www.coupang.com/vp/products/6060667011?isAddedCart=',
    r'https://www.coupang.com/vp/products/2530237?isAddedCart=',
    r'https://www.coupang.com/vp/products/266030794?isAddedCart=',
    r'https://www.coupang.com/vp/products/4978269600?isAddedCart='
]
price_list = []


In [ ]:
#LOOP THROUGH URLS AND SCRAPE PRICES
for url in url_list:
    driver = webdriver.Edge(driver_path_edge)

    try:
        driver.get(url)
        sleep(0.5)
        price = get_price(driver)
        time_utc_now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
        image_link = get_image(driver)
        link = url
        product_id = url.split(link_prefix)[1].split(link_sufix_split)[0]
        product_link = link_prefix + product_id + link_sufix
        data = str(price) + ",'" + time_utc_now + "','" + image_link + "','" + product_id + "','" + product_link + "'"

        act_dict = {
            'price': price,
            'time_utc_now': time_utc_now,
            'image_link': image_link,
            'product_id': product_id,
            'product_link': product_link
            }
        price_list.append(act_dict)
        
        dbf.insert_data(conn, cur, table_name, data)
    except:
        pass

    driver.quit()
    sleep(1)
price_list

In [ ]:
#CLOSE CONNECTION TO DATABASE
dbf.close_connection(conn)
